In [1]:
import pandas as pd

In [2]:
def setup():
    return pd.read_csv("ELO.csv")

In [3]:
def updateELO(rating, teamA, teamB, sA, sB):
    
    qA = 10**(rating[teamA] / 400)
    qB = 10**(rating[teamB] / 400)
    
    eA = qA/(qA + qB)
    eB = qB/(qA + qB)
    
    rating[teamA] = rating[teamA] + 20 (sA - eA)
    rating[teamB] = rating[teamB] + 20 (sB - eB)
    
    return rating

In [6]:
def readScore(rating, file):
    scores = open(file)
    osA = 0
    osB = 0
    lastA = ""
    for st in scores:
        [A, B] = st.split(",")
        [teamA, scoreA] = A.split("-")
        [teamB, scoreB] = B.split("-")

        if( lastA != "" and lastA != teamA):
            osA = 0
            osB = 0
        
        scoreA = int(scoreA)
        scoreB = int(scoreB)
        
        if( scoreA > scoreB):
            osA = osA + 1
        elif( scoreB > scoreA):
            osB = osB + 1
        else:
            osA = osA + .5
            osB = osB + .5
        
        lastA = teamA
    
    return rating

In [33]:
rating = setup()
other = rating
rating = pd.concat([rating, other]).reset_index()
del rating["index"]
rating

,ATL,BOS,CHD,DAL,FLA,GLA,GZC,HOU,HZS,LDN,NYE,PAR,PHI,SFS,SEO,SHD,TOR,VAL,VAN,WAS
0,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500
1,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500


In [7]:
rating = setup()
st = "ABC"
while( st != "DONE."):
    print("Please enter the name of the file containing new game scores to parse. \n  Enter \"DONE.\" to exit.")
    st = input("Filename: ")
    
    readScore(rating, st)
    
    print("File complete.")

Please enter the name of the file containing new game scores to parse. 
  Enter "DONE." to exit.
Filename: f


FileNotFoundError: [Errno 2] No such file or directory: 'f'